In [87]:
import numpy as np
import pandas as pd
import re, os, random

In [48]:
notes = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
note_to_num = {}
for num, note in enumerate(notes):
    note_to_num[note] = num
num_to_note = dict([[v,k] for k,v in note_to_num.items()])
same_note = {'A#':'Bb', 'C#':'Db', 'D#':'Eb', 'F#': 'Gb', 'G#':'Ab'}

def split_note(note):
    assert re.fullmatch('[A-G](#|b)?[0-7]', note) is not None, 'Note not formatted correctly: %s'%note
    return note[:-1], int(note[-1])

def shift_note(note, amount):
    # note taken in as string, amount is any integer
    # probably not needed until actually generating stuff
    note, octave = split_note(note)
    if note in same_note:
        note = same_note[note]
    new_num = note_to_num[note] + amount
    if new_num > 11:
        octave += 1
    elif new_num < 0:
        octave -= 1
    return num_to_note[(new_num) % 12] + str(octave)

def note_dist(note1, note2):
    # positive if note2 is above note1, 0 if same
    note1, octave1 = split_note(note1)
    note2, octave2 = split_note(note2)
    if note1 in same_note:
        note1 = same_note[note1]
    if note2 in same_note:
        note2 = same_note[note2]
    tot = (octave2 - octave1) * 12
    tot += note_to_num[note2] - note_to_num[note1]
    return tot

### Finding Slope Bounds

In [66]:
def find_slope_bounds(lst):
    max_jump, min_jump = 0, 0
    for i in range(len(lst) - 1):
        max_jump = max(max_jump, note_dist(lst[i], lst[i+1]))
        min_jump = min(min_jump, note_dist(lst[i], lst[i+1]))
    return str(min_jump) + ' ' + str(max_jump)

### Average Max Slope

In [119]:
def avgmaxslope(exp):
    max_asc, max_desc = 0,0
    for i in range(len(exp) - 1):
        print(exp[i][0], exp[i+1][0])
        if note_dist(exp[i][0], exp[i+1][0]) >= 0:
            max_asc = max(max_asc, note_dist(exp[i][0], exp[i+1][0]))
        if note_dist(exp[i][0], exp[i+1][0]) <= 0:
            max_desc = max(max_desc, abs(note_dist(exp[i][0], exp[i+1][0])))
    avg = (max_asc + max_desc) / 2
    return avg

#test = [('C4', 0.5), ('G4', 0.25), ('Bb5', 0.4)]
#note_dist('C4', 'G4')
#avgmaxslope(test)



C4 G4
G4 Bb5


7.5

### Selecting Notes

In [124]:
def select_note(lst, curr, minimum, maximum):
    #1: output of the possible notes
    #2: current note ('C4')
    #3: minimum slope
    #4: maximum slope
    #output single note from filtered list; else random of unfiltered list
    master_lst, lower_octave, equal_octave, higher_octave = [], [], [], []
    for key in lst:
        lower_octave.append(split_note(key)[0] + str((split_note(key)[1]) - 1))
        equal_octave.append(split_note(key)[0] + str(split_note(key)[1]))
        higher_octave.append(split_note(key)[0] + str((split_note(key)[1]) + 1))
    for value in lower_octave:
        if note_dist(value, curr) >= minimum:
            master_lst.append(value)
    for value in equal_octave:
        if note_dist(value, curr) >= minimum:
            master_lst.append(value)
        if note_dist(value, curr) <= maximum:
            master_lst.append(value)
    for value in lower_octave:
        if note_dist(value, curr) <= maximum:
            master_lst.append(value)
    if len(master_lst) == 0:
        return random.choice(lst)
    else:
        return random.choice(master_lst)

In [164]:
#TESTS


#key = 'C4'
#print(split_note(key))
#print(split_note(key)[0])
#print(split_note(key)[1])
#value = (split_note(key)[0] + str((split_note(key)[1]) - 1))
#print(value)

#select_note(['G3', 'F#3', 'A3', 'A#3'], 'C4', 3, 3)

'F#3'

### Determining Consonance

In [86]:
def consonance(measure): #measure is 1 measure of an s-expression
    total = 0.0
    i = 0
    for note in measure:
        if note[0][0] == "R":
            total += (0.1 * 1/float(note[0][1])) #note[0][1] is duration
        if note[0][0] == "C":
            total += (0.8 * 1/float(note[0][1]))
        if note[0][0] == "L":
            total += (0.4 * 1/float(note[0][1]))
        if note[0][0] == "X":
            total += (0.1 * 1/float(note[0][1]))
        if note[0][0] == "A":
            total += (0.6 * 1/float(note[0][1]))
        i += 1
    return total #total consonance value for that measure

#test = [('C8', 0.5), ('C8', 0.9), ('C8', 0.9)]
#print(consonance(test)



### Creating the S-Expressions Table

In [4]:
def categorize_note(note, chord, last_chord):
    return np.random.choice(['C', 'H', 'R'])

In [16]:
def create_s_exp(notes):
    # notes is list of tupes of (note_string, duration)
    s = ''
    notes_only = []
    for note, duration, chord, last_chord in notes:
        s += categorize_note(note, chord, last_chord) + '%.3f '%duration
        notes_only.append(note)
    return find_slope_bounds(notes_only) + ' ' + s

In [17]:
s_exp = pd.DataFrame(columns=['id', 'exp', 'song_id', 'song_index'])
s_exp = s_exp.set_index(['id'])
directory = 'midi_to_csv/' # 'raw_solos/'
song_num = 0
for filename in os.listdir(directory):
    if filename.endswith('Night.csv'):
        measure = 0
        last_chord = None
        curr_s_exp = []
        song = pd.read_csv(directory + filename)
        for i in range(len(song)):
            curr_note = song.iloc[i]
            if measure != int(curr_note['start_time']):
                s = create_s_exp(curr_s_exp)
                row = {'exp': s, 'song_id': song_num, 'song_index': measure}
                s_exp = s_exp.append(row, ignore_index=True)
                curr_s_exp = []
                measure = int(curr_note['start_time'])
            curr_s_exp.append((curr_note['note_name'], curr_note['duration'], None, None))#curr_note['chord'], last_chord))
            #last_chord = curr_note['chord']
        s = create_s_exp(curr_s_exp)
        row = {'exp': s, 'song_id': song_num, 'song_index': measure}
        s_exp = s_exp.append(row, ignore_index=True)
        song_num += 1
song_num

1

In [20]:
s_exp.to_csv('test_files/s_exp_test.csv')